## This notebook is for evaluating the performance of each server from the 3 servers ##
1. MySQL
2. MSSQL (Microsoft SQL Server)
3. PostgreSQL

### Importing the required libraries ###

In [36]:
import time
from dotenv import load_dotenv
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

### Setting the dataframe that will contain the results ###

In [37]:
servers_performance = pd.DataFrame(columns=["MySQL/INSERT", "MySQL/UPDATE", "MSSQL/INSERT", "MSSQL/UPDATE", "PostgreSQL/INSERT", "PostgreSQL/UPDATE"],
                                   index=range(1, 101))
servers_performance.index.name = "StatementID"

In [38]:
servers_performance

,MySQL/INSERT,MySQL/UPDATE,MSSQL/INSERT,MSSQL/UPDATE,PostgreSQL/INSERT,PostgreSQL/UPDATE
StatementID,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
96,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN


### Choose 2 statements to evaluate the performance using them ###

In [39]:
add_record = """INSERT INTO Products VALUES('P1', 1500, 100);"""

update_record = """UPDATE Products SET ItemsInStock = 95 WHERE ProductID = 'P1';"""

### MySQL Server ###

In [40]:
load_dotenv()
ENV_MYSQL_USER = os.getenv('MYSQL_USER')
ENV_MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
ENV_MYSQL_HOST = os.getenv('MYSQL_HOST')
ENV_MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')

In [41]:
import mysql.connector as connector

# Connect to the database
connection = connector.connect(user=ENV_MYSQL_USER,
                        password=ENV_MYSQL_PASSWORD,
                        host=ENV_MYSQL_HOST,
                        database=ENV_MYSQL_DATABASE)

In [42]:
# Create a link "cursor" between MySQL and Python
cursor = connection.cursor()

In [43]:
for i in range(100):
    # inserting performance
    start_time = time.time()
    cursor.execute(add_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 0] = total_time_in_msec

    # updating performance
    start_time = time.time()
    cursor.execute(update_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 1] = total_time_in_msec

    cursor.execute("""DELETE FROM Products;""")


In [44]:
cursor.close()
connection.close()

### MSSQL ###

In [45]:
load_dotenv()
ENV_MSSQL_SERVER = os.getenv('MSSQL_SERVER')

ENV_MSSQL_DATABASE = os.getenv('MSSQL_DATABASE')

In [46]:
import pyodbc

# Connect to the database using Windows authentication
connection = pyodbc.connect('DRIVER={SQL Server};SERVER='+ENV_MSSQL_SERVER+';DATABASE='+ENV_MYSQL_DATABASE+';Trusted_Connection=yes;')

In [47]:
# Create a link "cursor" between MSSQL and Python
cursor = connection.cursor()

In [48]:
for i in range(100):
    # inserting performance
    start_time = time.time()
    cursor.execute(add_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 2] = total_time_in_msec

    # updating performance
    start_time = time.time()
    cursor.execute(update_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 3] = total_time_in_msec

    cursor.execute("""DELETE FROM Products;""")


In [49]:
cursor.close()
connection.close()

### PostgreSQL ###

In [50]:
load_dotenv()
ENV_POSTGRES_USER = os.getenv('POSTGRES_USER')
ENV_POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
ENV_POSTGRES_HOST = os.getenv('POSTGRES_HOST')
ENV_POSTGRES_DATABASE = os.getenv('POSTGRES_DATABASE')
ENV_POSTGRES_PORT = os.getenv('POSTGRES_PORT')

In [51]:
import psycopg2

# Connect to the database
connection = psycopg2.connect(host=ENV_POSTGRES_HOST,
                                port=ENV_POSTGRES_PORT,
                                database=ENV_POSTGRES_DATABASE,
                                user=ENV_POSTGRES_USER,
                                password=ENV_POSTGRES_PASSWORD)

In [52]:
# Create a link "cursor" between PostgreSQL and Python
cursor = connection.cursor()

In [53]:
for i in range(100):
    # inserting performance
    start_time = time.time()
    cursor.execute(add_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 4] = total_time_in_msec

    # updating performance
    start_time = time.time()
    cursor.execute(update_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 5] = total_time_in_msec

    cursor.execute("""DELETE FROM Products;""")

### Statistics ###

In [54]:
servers_performance

,MySQL/INSERT,MySQL/UPDATE,MSSQL/INSERT,MSSQL/UPDATE,PostgreSQL/INSERT,PostgreSQL/UPDATE
StatementID,,,,,,
1,2.989292,0.0,14.630556,110.212088,2.991915,1.996517
2,0.0,0.0,0.322342,0.329971,0.0,0.997305
3,0.0,0.991821,0.0,0.0,0.0,0.0
4,0.0,0.998497,0.0,0.0,0.0,0.996828
5,0.0,1.000881,0.999928,0.0,0.0,0.997782
...,...,...,...,...,...,...
96,0.0,0.0,0.998259,0.0,0.227213,0.306129
97,0.989199,0.0,0.0,0.0,0.221491,0.294685
98,0.229836,0.302553,0.0,0.997066,0.216246,0.136852


In [55]:
servers_performance.describe()

,MySQL/INSERT,MySQL/UPDATE,MSSQL/INSERT,MSSQL/UPDATE,PostgreSQL/INSERT,PostgreSQL/UPDATE
count,100.0,100.0,100.0,100.0,100.0,100.0
unique,43.0,42.0,37.0,29.0,41.0,45.0
top,0.0,0.0,0.0,0.0,0.0,0.0
freq,54.0,56.0,60.0,65.0,55.0,48.0


In [56]:
servers_performance.mean()

MySQL/INSERT         0.333397
MySQL/UPDATE         0.317361
MSSQL/INSERT         0.359352
MSSQL/UPDATE         1.281676
PostgreSQL/INSERT    0.287201
PostgreSQL/UPDATE    0.379808
dtype: float64

In [57]:
servers_performance.max()

MySQL/INSERT           2.989292
MySQL/UPDATE             1.1127
MSSQL/INSERT          14.630556
MSSQL/UPDATE         110.212088
PostgreSQL/INSERT      2.991915
PostgreSQL/UPDATE      1.996517
dtype: object

In [58]:
servers_performance.min()

MySQL/INSERT         0.0
MySQL/UPDATE         0.0
MSSQL/INSERT         0.0
MSSQL/UPDATE         0.0
PostgreSQL/INSERT    0.0
PostgreSQL/UPDATE    0.0
dtype: object